## Problem Set 9 Template

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import statsmodels.api as sm
from pandas_datareader import DataReader as pdr

Read dataset of industry returns

In [ ]:
df = pd.read_csv('https://www.dropbox.com/s/yoficjiwlm88qe4/ps_09_industry_returns.csv?dl=1')
df['Date'] = pd.to_datetime(df['Date']).dt.to_period('M')
df = df.set_index('Date')
df.iloc[:12,:10]

Calculate rolling average means using last 12 months of data

In [ ]:
WINDOW = 12
avgs = df.rolling(WINDOW).mean()    # calculate 12-month rolling average
avgs = avgs.dropna()                # drop any rows with missing rolling averages
avgs.iloc[:10,:10]                  # display 1st 10 rows and columns of dataframe

Sort into quintiles each month

In [4]:
def cut_quintiles(x):
    try:
        out = pd.qcut(x, 5, labels=["Lo", "Q2", "Q3", "Q4", "Hi"])
    except:
        out = pd.Series(np.nan, index=x.index)
    return out

In [ ]:
ports = avgs.apply(cut_quintiles, axis=1)   # assign industry to a quintile based on each row (axis=1 applies the function row-by-row)
ports.iloc[:10,:10]                         # display 1st 10 rows and columns of dataframe

Calculate portfolio weights for 'winner' portfolio

In [ ]:
hi = (ports=='Hi')*1.0              # create a dataframe with 1's for entries equal to 'Hi' and 0's elsewhere
sum_wgts = hi.sum(axis=1)           # sum over the weights each month (axis=1 applies sum function row-by-row)
hi = hi.div(sum_wgts, axis=0)       # divide by the sum of the weights each month to create portfolio weights summing to 1
hi.iloc[:10,:10]                    # display 1st 10 rows and columns of dataframe

Calculate portfolio weights for 'loser' portfolio

In [ ]:
lo = (ports=='Lo')*1.0              # create a dataframe with 1's for entries equal to 'Lo' and 0's elsewhere
sum_wgts = lo.sum(axis=1)           # sum over the weights each month (axis=1 applies sum function row-by-row)
lo = lo.div(sum_wgts, axis=0)       # divide by the sum of the weights each month to create portfolio weights summing to 1
lo.iloc[:10,:10]                    # display 1st 10 rows and columns of dataframe

Calculate time-series of returns for each portfolio

(Note: the rolling average return included the current period's return, so we need to skip a month when applying the weights)

In [ ]:
#  Multiply returns by weights calculated in prior month
rets = pd.DataFrame(dtype=float,columns=['hi','lo'],index=hi.index[1:])     # create a dataframe to store time-series of 'Hi' and 'Lo' returns; 1st month is one-month after first 12-month rolling average
for d in rets.index:                                                        # loop over dates
    rets.loc[d,'hi'] = hi.loc[d-1] @ df.loc[d]                              # for date d, calculate the 'Hi' portfolio return by summing the product of the industry's portfolio weight by its realized return
    rets.loc[d,'lo'] = lo.loc[d-1] @ df.loc[d]                              # for date d, calculate the 'Lo' portfolio return by summing the product of the industry's portfolio weight by its realized return
rets

Visualize the cumulative returns

In [ ]:
# Plot cumulative returns of the past winners and past losers
fig = go.Figure()

string =  "Strategy: Buy Past Winners<br>"
string += "Date: %{x}<br>"
string += "FV of $1: $%{y:,.2f}<br>"
string += "<extra></extra>"
trace_hi  = go.Scatter(x=rets.index.to_timestamp('M'), y=(1+rets.hi).cumprod(), mode="lines", name='Buy Winners', hovertemplate=string)
fig.add_trace(trace_hi)

string =  "Strategy: Buy Past Losers<br>"
string += "Date: %{x}<br>"
string += "FV of $1: $%{y:,.2f}<br>"
string += "<extra></extra>"
trace_lo= go.Scatter(x=rets.index.to_timestamp('M'), y=(1+rets.lo).cumprod(), mode="lines", name='Sell Losers',hovertemplate=string)
fig.add_trace(trace_lo)
#Formatting
fig.update_yaxes(type="log", title="FV of $1")
fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01),
                  yaxis = dict(tickmode = 'array',tickvals = [2, 10, 100, 1000, 2500],)    
                  )
fig.show()

Merge in market excess return and risk-free rate & calculate excess returns

In [ ]:
# Portfolio alpha and beta
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
rets = rets.join(ff3[['Mkt-RF','RF']], how='left')
rets['hi-rf'] = rets['hi']-rets['RF']
rets['lo-rf'] = rets['lo']-rets['RF']
rets

Market model regressions to be completed by you below